In [1]:
import tensorlayer as tl
import tensorflow as tf
import rawpy

import model
import utils

from PIL import Image
import numpy as np
import glob
import importlib
from PIL import Image

## Set config

In [2]:
import config as _config
importlib.reload(_config)
from config import config, log_config

###====================== HYPER-PARAMETERS ===========================###
## Adam
batch_size = config.TRAIN.batch_size
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = config.TRAIN.n_epoch_init
## adversarial learning (SRGAN)
n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
training_dir = config.TRAIN.training_dir
pretrain_checkpoint = config.TRAIN.pretrain_checkpoint
train_data_dir = config.TRAIN.train_data_dir
dark_model_dir = config.TRAIN.dark_model_dir #change

crop_num = config.TRAIN.crop_num

tl.files.exists_or_mkdir(training_dir)
tl.files.exists_or_mkdir(pretrain_checkpoint)

[TL] [!] training_dir exists ...
[TL] [!] checkpoint exists ...


True

## Define Model

In [20]:
import model as _model
importlib.reload(_model)
import utils as _utils
importlib.reload(_utils)
import dark_utils as _dark_utils
importlib.reload(_dark_utils)

from model import *
from utils import *
from dark_utils import *

In [4]:
t_image = tf.placeholder('float32', [None, 96, 96, 3], name='t_image_input_to_SRGAN_generator')
t_target_image = tf.placeholder('float32', [None, 384, 384, 1], name='t_target_image')

net_g = SRGAN_g(t_image, is_train=True, reuse=False)
net_d, logits_real = SRGAN_d(t_target_image, is_train=True, reuse=False)
_, logits_fake = SRGAN_d(net_g.outputs, is_train=True, reuse=True)

# net_g.print_params(False)￼

net_g.print_params(False)
# net_g.print_layers()
net_d.print_params(False)
# net_d.print_layers()

## test inference
net_g_test = SRGAN_g(t_image, is_train=False, reuse=True)

[TL] InputLayer  SRGAN_g/in: (?, 96, 96, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/

[TL] ElementwiseLayer SRGAN_g/b_residual_add/15: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/add3: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out_new: n_filter: 1 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh
[TL] WARNING: Function: `tensorlayer.layers.utils.set_name_reuse` (in file: c:\python_3.6.7\lib\site-packages\tensorlayer\layers\utils.py)

[TL]   param  23: SRGAN_g/n64s1/b1/2/beta:0 (64,)              float32_ref
[TL]   param  24: SRGAN_g/n64s1/b1/2/gamma:0 (64,)              float32_ref
[TL]   param  25: SRGAN_g/n64s1/b1/2/moving_mean:0 (64,)              float32_ref
[TL]   param  26: SRGAN_g/n64s1/b1/2/moving_variance:0 (64,)              float32_ref
[TL]   param  27: SRGAN_g/n64s1/c2/2/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param  28: SRGAN_g/n64s1/b2/2/beta:0 (64,)              float32_ref
[TL]   param  29: SRGAN_g/n64s1/b2/2/gamma:0 (64,)              float32_ref
[TL]   param  30: SRGAN_g/n64s1/b2/2/moving_mean:0 (64,)              float32_ref
[TL]   param  31: SRGAN_g/n64s1/b2/2/moving_variance:0 (64,)              float32_ref
[TL]   param  32: SRGAN_g/n64s1/c1/3/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param  33: SRGAN_g/n64s1/b1/3/beta:0 (64,)              float32_ref
[TL]   param  34: SRGAN_g/n64s1/b1/3/gamma:0 (64,)              float32_ref
[TL]   param  35: SRGAN_g/n64s1/b1/3/moving_mean:0 (64,) 

[TL]   param 126: SRGAN_g/n64s1/b1/12/moving_variance:0 (64,)              float32_ref
[TL]   param 127: SRGAN_g/n64s1/c2/12/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 128: SRGAN_g/n64s1/b2/12/beta:0 (64,)              float32_ref
[TL]   param 129: SRGAN_g/n64s1/b2/12/gamma:0 (64,)              float32_ref
[TL]   param 130: SRGAN_g/n64s1/b2/12/moving_mean:0 (64,)              float32_ref
[TL]   param 131: SRGAN_g/n64s1/b2/12/moving_variance:0 (64,)              float32_ref
[TL]   param 132: SRGAN_g/n64s1/c1/13/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 133: SRGAN_g/n64s1/b1/13/beta:0 (64,)              float32_ref
[TL]   param 134: SRGAN_g/n64s1/b1/13/gamma:0 (64,)              float32_ref
[TL]   param 135: SRGAN_g/n64s1/b1/13/moving_mean:0 (64,)              float32_ref
[TL]   param 136: SRGAN_g/n64s1/b1/13/moving_variance:0 (64,)              float32_ref
[TL]   param 137: SRGAN_g/n64s1/c2/13/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 138: SRGAN_g/n64s1/b

[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/1: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/1: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/2: decay: 0.900000 epsilon: 0.000010 act: relu is

[TL] ElementwiseLayer SRGAN_g/add3: size: (?, 96, 96, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2_new: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2_new: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out_new: n_filter: 1 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


In [5]:
t_raw_for_dark = tf.placeholder(tf.float32, [None, None, None, 4]) #change
out_image = dark_network(t_raw_for_dark)

## Define training op

In [6]:
d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake), name='d2')
d_loss = d_loss1 + d_loss2

g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake), name='g')
mse_loss = tl.cost.mean_squared_error(net_g.outputs, t_target_image, is_mean=True)
# vgg_loss = 2e-6 * tl.cost.mean_squared_error(vgg_predict_emb.outputs, vgg_target_emb.outputs, is_mean=True)

# g_loss = mse_loss + vgg_loss + g_gan_loss
g_loss = mse_loss + g_gan_loss

g_vars = tl.layers.get_variables_with_name('SRGAN_g', True, True)
d_vars = tl.layers.get_variables_with_name('SRGAN_d', True, True)

with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr_init, trainable=False)
## Pretrain
g_optim_init = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(mse_loss, var_list=g_vars)
## SRGAN
g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars)
d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars)

[TL]   [*] geting variables with SRGAN_g
[TL]   got   0: SRGAN_g/n64s1/c/kernel:0   (3, 3, 3, 64)
[TL]   got   1: SRGAN_g/n64s1/c/bias:0   (64,)
[TL]   got   2: SRGAN_g/n64s1/c1/0/kernel:0   (3, 3, 64, 64)
[TL]   got   3: SRGAN_g/n64s1/b1/0/beta:0   (64,)
[TL]   got   4: SRGAN_g/n64s1/b1/0/gamma:0   (64,)
[TL]   got   5: SRGAN_g/n64s1/c2/0/kernel:0   (3, 3, 64, 64)
[TL]   got   6: SRGAN_g/n64s1/b2/0/beta:0   (64,)
[TL]   got   7: SRGAN_g/n64s1/b2/0/gamma:0   (64,)
[TL]   got   8: SRGAN_g/n64s1/c1/1/kernel:0   (3, 3, 64, 64)
[TL]   got   9: SRGAN_g/n64s1/b1/1/beta:0   (64,)
[TL]   got  10: SRGAN_g/n64s1/b1/1/gamma:0   (64,)
[TL]   got  11: SRGAN_g/n64s1/c2/1/kernel:0   (3, 3, 64, 64)
[TL]   got  12: SRGAN_g/n64s1/b2/1/beta:0   (64,)
[TL]   got  13: SRGAN_g/n64s1/b2/1/gamma:0   (64,)
[TL]   got  14: SRGAN_g/n64s1/c1/2/kernel:0   (3, 3, 64, 64)
[TL]   got  15: SRGAN_g/n64s1/b1/2/beta:0   (64,)
[TL]   got  16: SRGAN_g/n64s1/b1/2/gamma:0   (64,)
[TL]   got  17: SRGAN_g/n64s1/c2/2/kernel:0  

## restore model

In [7]:
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True,
                                        log_device_placement=False))

In [8]:
tl.layers.initialize_global_variables(sess)
if tl.files.load_and_assign_npz(sess=sess, name=training_dir + '/g_srgan.npz', network=net_g) is None:
    load_pretrain_model(sess=sess, npz_file=pretrain_checkpoint + '/g_srgan.npz', network=net_g)
tl.files.load_and_assign_npz(sess=sess, name=training_dir + '/d_srgan.npz', network=net_d)

[TL] WARNING: Function: `tensorlayer.layers.utils.initialize_global_variables` (in file: c:\python_3.6.7\lib\site-packages\tensorlayer\layers\utils.py) is deprecated and will be removed after 2018-09-30.
Instructions for updating: This API is deprecated in favor of `tf.global_variables_initializer`

[TL] ERROR: file training_dir/g_srgan.npz doesn't exist.
[TL] [*] Load checkpoint/g_srgan.npz SUCCESS!
[TL] ERROR: file training_dir/d_srgan.npz doesn't exist.


In [9]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='DARK')[0].name

'DARK/g_conv1_1/weights:0'

In [21]:
var_dict = dark_model_var_dict(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='DARK'))

In [14]:
var_dict

{'g_conv1_1/weights:0': <tf.Variable 'DARK/g_conv1_1/weights:0' shape=(3, 3, 4, 32) dtype=float32_ref>,
 'g_conv1_1/biases:0': <tf.Variable 'DARK/g_conv1_1/biases:0' shape=(32,) dtype=float32_ref>,
 'g_conv1_2/weights:0': <tf.Variable 'DARK/g_conv1_2/weights:0' shape=(3, 3, 32, 32) dtype=float32_ref>,
 'g_conv1_2/biases:0': <tf.Variable 'DARK/g_conv1_2/biases:0' shape=(32,) dtype=float32_ref>,
 'g_conv2_1/weights:0': <tf.Variable 'DARK/g_conv2_1/weights:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 'g_conv2_1/biases:0': <tf.Variable 'DARK/g_conv2_1/biases:0' shape=(64,) dtype=float32_ref>,
 'g_conv2_2/weights:0': <tf.Variable 'DARK/g_conv2_2/weights:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 'g_conv2_2/biases:0': <tf.Variable 'DARK/g_conv2_2/biases:0' shape=(64,) dtype=float32_ref>,
 'g_conv3_1/weights:0': <tf.Variable 'DARK/g_conv3_1/weights:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 'g_conv3_1/biases:0': <tf.Variable 'DARK/g_conv3_1/biases:0' shape=(128,) dtype=float32_ref>,
 'g_

In [23]:
#change load the pre-train dark model
saver = tf.train.Saver(var_dict)
ckpt = tf.train.get_checkpoint_state(dark_model_dir)
if ckpt:
    print('loaded ' + ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)

loaded dark_model/Sony\model.ckpt
INFO:tensorflow:Restoring parameters from dark_model/Sony\model.ckpt


In [18]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
# import os
# checkpoint_path = os.path.join(model_dir, "model.ckpt")

# List ALL tensors example output: v0/Adam (DT_FLOAT) [3,3,1,80]
print_tensors_in_checkpoint_file(file_name=dark_model_dir + '/model.ckpt', tensor_name='', all_tensors=True)

tensor_name:  Variable
[[[[ 0.01418491  0.01014553 -0.0381809  ... -0.01492392 -0.00561918
     0.00365046]
   [ 0.00640314  0.00187061 -0.00763831 ... -0.00619096  0.02731778
    -0.00915546]
   [-0.00765199 -0.01647181  0.01573027 ...  0.02120006 -0.00121986
     0.01333133]
   ...
   [-0.0209457  -0.01826328 -0.0185713  ... -0.01652458  0.00732991
     0.00369776]
   [-0.01112286  0.02601658 -0.02714815 ... -0.01312493  0.0285363
    -0.00610322]
   [ 0.0153092  -0.00487628  0.01463819 ...  0.03717678 -0.03932474
     0.00670664]]

  [[ 0.02339673  0.01710775 -0.03070614 ... -0.00217706 -0.02632207
     0.0027227 ]
   [ 0.0327966   0.01473661  0.02650706 ... -0.00852472  0.00558816
     0.01025653]
   [-0.01506554  0.03036044  0.00258343 ... -0.017749   -0.01851437
    -0.03286755]
   ...
   [ 0.02551023 -0.01574194  0.00115547 ... -0.00877026  0.00206622
    -0.00675561]
   [ 0.00689809 -0.00727433  0.00622564 ... -0.01294741 -0.01113787
    -0.02579891]
   [ 0.01047765  0.00304267

     9.44643933e-03  3.11753675e-02]]]]
tensor_name:  beta1_power
0.0
tensor_name:  beta2_power
0.0
tensor_name:  g_conv10/biases
[0.02902106 0.0132251  0.01509992 0.01121732 0.02279354 0.00109669
 0.01380065 0.01765538 0.00015123 0.01221313 0.00702888 0.00989802]
tensor_name:  g_conv10/biases/Adam
[ 3.5600335e-04 -1.1580939e-03  4.1455594e-03 -1.1512339e-04
 -1.6152174e-03  3.8894536e-03  9.8007426e-05 -1.5579703e-03
  4.7836197e-03  9.5322430e-05 -1.4693062e-03  4.0301280e-03]
tensor_name:  g_conv10/biases/Adam_1
[0.00021895 0.00027065 0.00028853 0.00023835 0.00026841 0.00027654
 0.00023169 0.00026769 0.00027519 0.00024776 0.00026594 0.00026613]
tensor_name:  g_conv10/weights
[[[[-0.25022227 -0.00576119 -0.07352486 -0.16310538 -0.0981524
    -0.35527626 -0.13424042 -0.1261601  -0.11293742 -0.31947163
     0.29186815  0.34601298]
   [ 0.00538841  0.23425163  0.02968764  0.23728663  0.25244313
    -0.10955855 -0.17123008 -0.02928336 -0.09632403  0.1398968
     0.06661507 -0.08568716]
 

     1.59434669e-04  3.79131699e-04 -5.14226267e-04 -5.62421556e-05]]]]
tensor_name:  g_conv10/weights/Adam_1
[[[[2.23235043e-06 2.21794244e-06 2.16903049e-06 2.37157565e-06
    2.20373408e-06 2.14437136e-06 2.25448412e-06 2.22798349e-06
    2.19851586e-06 2.49051459e-06 2.20225911e-06 2.16286412e-06]
   [3.71971910e-05 3.66875429e-05 3.54495969e-05 3.92980910e-05
    3.64584666e-05 3.51131348e-05 3.72937393e-05 3.69749941e-05
    3.60676895e-05 4.12360678e-05 3.64781226e-05 3.52669340e-05]
   [2.51996653e-07 2.89965072e-07 2.66369710e-07 2.69999134e-07
    2.87761765e-07 2.58986262e-07 2.57845073e-07 2.89338004e-07
    2.63734591e-07 2.83147642e-07 2.86215567e-07 2.56591136e-07]
   [1.06736134e-05 1.37491706e-05 1.19713868e-05 1.14313507e-05
    1.37148127e-05 1.13455826e-05 1.12300413e-05 1.38763126e-05
    1.15719922e-05 1.20988389e-05 1.37401321e-05 1.09143775e-05]
   [1.17223237e-06 1.13071565e-06 1.11161364e-06 1.24099552e-06
    1.12263240e-06 1.10335725e-06 1.17540162e-06 1.136

    -1.46579251e-01 -1.43612325e-01]]]]
tensor_name:  g_conv1_1/weights/Adam
[[[[ 3.42093786e-04  1.10700443e-04 -1.32487447e-04 ... -3.40564206e-04
    -2.60787638e-04 -4.19968070e-04]
   [ 4.21462115e-04 -3.56719509e-04 -3.28010414e-04 ... -5.02959127e-04
    -2.73133162e-04 -5.22483373e-04]
   [ 6.26622641e-05  5.94070880e-04 -3.89524910e-04 ... -5.85372945e-05
    -6.98309916e-04 -4.31297522e-04]
   [ 4.21076344e-04 -1.98785070e-04 -3.22852284e-04 ... -4.68163227e-04
    -2.51637422e-04 -4.65221849e-04]]

  [[ 3.40825442e-04  2.00980518e-04 -2.31438811e-04 ... -3.67638801e-04
    -2.95976031e-04 -5.44601586e-04]
   [ 5.23169525e-04 -4.86184930e-04 -1.99921706e-04 ... -5.37450076e-04
    -2.13308987e-04 -5.14356827e-04]
   [ 1.06935986e-04  6.36804732e-04 -3.47798254e-04 ... -6.71284943e-05
    -7.17969902e-04 -4.60729265e-04]
   [ 5.65666007e-04 -3.76234355e-04 -3.36462486e-04 ... -5.02620533e-04
    -1.87208934e-04 -6.32980198e-04]]

  [[ 3.58539255e-04  1.06368883e-04 -1.27538020

  0.00907239 -0.00633285]
tensor_name:  g_conv1_2/biases/Adam
[ 4.8980755e-03 -1.8194779e-04 -1.6802769e-02 -7.3441947e-03
 -5.2106590e-03  8.4501764e-05  4.9908916e-03 -6.2383927e-04
 -2.6140066e-02 -1.4219566e-03  3.0429871e-03  1.6432968e-03
  9.0767245e-04 -7.8254817e-03 -1.8220931e-02 -4.4394839e-03
 -3.8447951e-03 -8.7985769e-04  5.8776890e-03 -3.7885900e-03
  3.5807099e-03  8.4176725e-03  1.2718557e-02 -2.8598399e-03
 -1.0197996e-02 -2.8035754e-02 -5.6582084e-04 -1.0208522e-03
  9.6736746e-03 -1.0478335e-02  3.7463328e-03 -1.0407950e-02]
tensor_name:  g_conv1_2/biases/Adam_1
[1.78393268e-03 7.67960519e-05 2.46555358e-03 1.70425475e-02
 7.49120722e-03 6.59817815e-05 5.48906683e-04 8.63178109e-04
 2.09795106e-02 4.24648914e-03 2.67605676e-04 5.40562323e-05
 5.29284170e-03 2.22323090e-03 6.47703279e-03 2.42691720e-03
 5.33574494e-04 1.03673055e-05 4.99660615e-04 8.77191138e-04
 1.56603972e-04 2.36343080e-03 5.98722603e-03 1.40597008e-03
 1.28546357e-02 9.08502098e-03 1.20752037e-03

...

 1.4612043e-07 9.1532144e-08 3.4436115e-07 4.2209816e-08]
tensor_name:  g_conv8_2/weights
[[[[-0.06080032  0.04280667  0.04706988 ... -0.05957636  0.0144258
     0.06785566]
   [-0.03465157 -0.02400365 -0.03640243 ... -0.01454898  0.00322112
     0.00794708]
   [ 0.07014129  0.05456392 -0.00505122 ... -0.0316001   0.03783866
     0.03625356]
   ...
   [-0.10858382  0.00727159  0.01171897 ... -0.05324809  0.00073088
    -0.03106004]
   [ 0.00718663  0.03328753 -0.0321922  ... -0.00145446 -0.06565911
     0.06065911]
   [-0.02262369 -0.09565922 -0.04829492 ...  0.05593719  0.02272467
    -0.05369668]]

  [[ 0.02967864 -0.03145485  0.03866523 ...  0.01004828 -0.08216216
     0.11116562]
   [-0.05841044  0.01627929  0.03890869 ...  0.06645483  0.01360148
    -0.0477814 ]
   [ 0.01230717  0.1354987  -0.00219754 ... -0.02826586  0.00383572
     0.09373645]
   ...
   [ 0.01492966  0.01766367 -0.01480714 ...  0.05936226 -0.04198008
     0.05516372]
   [-0.06479298 -0.00421689  0.05180372 ... -

     1.26161240e-05 -1.18561456e-06]]]]
tensor_name:  g_conv8_2/weights/Adam_1
[[[[1.32927029e-08 2.22186908e-10 2.13587334e-08 ... 9.19405274e-10
    3.38389383e-09 9.53863211e-10]
   [1.53327839e-08 2.21905092e-10 6.92705182e-09 ... 4.79963003e-09
    1.15131726e-08 4.25548929e-09]
   [5.28533928e-09 3.05205930e-11 3.64179642e-09 ... 4.09206280e-10
    1.70125591e-09 3.27445238e-10]
   ...
   [1.18991972e-08 1.27194255e-10 1.19056383e-08 ... 3.39416850e-09
    1.52041366e-08 2.11601581e-09]
   [1.58406266e-08 3.77268841e-11 3.00279943e-08 ... 9.64919700e-10
    1.63861225e-09 6.13088025e-10]
   [3.89430355e-09 7.80562143e-11 2.98796055e-09 ... 6.25940244e-10
    2.57387822e-09 3.75621118e-10]]

  [[1.35329170e-08 2.29726030e-10 2.20971508e-08 ... 9.28436716e-10
    3.38913875e-09 9.94629268e-10]
   [1.57656572e-08 2.27375993e-10 7.01784320e-09 ... 4.94901453e-09
    1.16775025e-08 4.30696723e-09]
   [5.61000446e-09 3.01722605e-11 3.87790822e-09 ... 4.89770002e-10
    1.77464388e-09 3

     1.06889885e-02 -3.61110158e-02]]]]
tensor_name:  g_conv9_1/weights/Adam
[[[[-4.10790526e-05 -1.14495906e-05  1.75825016e-06 ... -1.32613013e-05
     3.02844346e-05 -1.97751524e-05]
   [-6.21881918e-06 -1.17800113e-07  3.88475019e-06 ...  5.96836035e-05
     6.71344387e-05 -2.62435642e-05]
   [-5.84913432e-05 -1.45811991e-05  8.23452581e-07 ...  1.19869750e-04
    -6.62342018e-06 -2.14635093e-05]
   ...
   [-8.40973735e-06 -1.20261539e-05 -1.23406937e-06 ...  7.40810137e-05
     1.46163205e-04 -5.04751260e-05]
   [ 2.94380311e-06  1.27362537e-05 -5.30082434e-06 ... -5.63837420e-05
    -5.01806026e-05  1.70358126e-05]
   [ 3.37432425e-06  6.78277911e-06 -2.83322925e-06 ... -1.27302674e-05
    -1.81832511e-05  1.07688847e-05]]

  [[-4.48206847e-05 -1.21698458e-05  5.59311411e-06 ... -1.43285615e-05
     2.16411427e-05 -3.34205038e-06]
   [-3.84023497e-06 -8.98599581e-07  3.18988373e-06 ...  6.12589356e-05
     4.95275453e-05  2.61899586e-05]
   [-5.92189754e-05 -1.57103605e-05 -1.269

    3.07383274e-08 2.32219244e-09]]]]
tensor_name:  g_conv9_2/biases
[-0.06268614  0.08813012 -0.03201216  0.02965593 -0.06957643  0.0759737
 -0.04209797 -0.04196005  0.00926075 -0.02473345  0.05216103  0.05498853
  0.06897036 -0.01811601 -0.0496945  -0.06815293  0.04258759  0.05894985
  0.05343451  0.06498443  0.08599438  0.06419683 -0.00154879 -0.00385741
  0.0766124  -0.0265874  -0.03784949 -0.01764609  0.09135457 -0.04289544
  0.08905654  0.07333904]
tensor_name:  g_conv9_2/biases/Adam
[-2.0238187e-04 -1.7305486e-03 -4.0908813e-04  2.4224664e-03
 -2.2353437e-04 -5.4032705e-04  3.5882732e-04 -2.4137604e-04
 -9.8287885e-04 -4.1696490e-04 -3.0977099e-04  4.7097323e-03
  1.0238300e-03 -9.5945506e-05 -3.3296918e-05  3.7408047e-04
  4.2885359e-04  2.2982794e-03 -1.0101132e-03  2.7133236e-03
 -1.5928834e-03 -1.4753193e-04  1.0433345e-04  1.2352377e-04
  1.0293643e-03  1.3954495e-04  4.2873682e-04 -2.8232709e-04
 -6.5560325e-04  1.8839604e-05  2.1160583e-03  2.1740139e-04]
tensor_name:  g_

    -3.72619470e-05 -5.62330315e-05]]]]
tensor_name:  g_conv9_2/weights/Adam_1
[[[[1.1730182e-08 1.8952174e-07 1.9279254e-09 ... 4.6916188e-10
    2.7422567e-08 6.0929658e-08]
   [4.0405994e-09 4.1318916e-08 9.9660902e-10 ... 2.9238539e-10
    2.3150518e-08 1.9031182e-08]
   [1.4541184e-08 1.5775232e-07 3.4781720e-09 ... 1.3125416e-10
    8.3969255e-08 7.7866083e-08]
   ...
   [1.3711974e-07 1.6862641e-06 3.6229757e-08 ... 2.0814821e-09
    8.8831649e-07 7.3892920e-07]
   [8.5346343e-08 1.1151259e-06 2.1147809e-08 ... 1.7106818e-09
    4.4495010e-07 4.3667163e-07]
   [3.0038059e-08 4.8051822e-07 4.4281441e-09 ... 9.0961921e-10
    5.6222103e-08 1.6470051e-07]]

  [[1.1723454e-08 1.8982664e-07 1.9524689e-09 ... 4.7168852e-10
    2.7505596e-08 6.0952331e-08]
   [4.1411821e-09 4.2263693e-08 1.0161317e-09 ... 3.8271489e-10
    2.3252550e-08 1.9131617e-08]
   [1.4657464e-08 1.5829937e-07 3.5237315e-09 ... 1.3340155e-10
    8.4572960e-08 7.8394962e-08]
   ...
   [1.3780776e-07 1.6965477e-06 

In [19]:
print_tensors_in_checkpoint_file??

## Training
### initialize G

In [15]:
sess.run(tf.assign(lr_v, lr_init))
print(" ** fixed learning rate: %f (for init G)" % lr_init)

 ** fixed learning rate: 0.000100 (for init G)


In [103]:
train_data_list = sorted(tl.files.load_file_list(train_data_dir, regx = '^0.*.ARW', printable = False))

## get a sub set of sample, test for training

In [107]:
sample_index = list(range(0, 320, 20))
sample_file_name = [train_data_list[i] for i in sample_index]

In [108]:
rgb_sample, raw_sample = get_inputs_labels(file_dir = train_data_dir,
                                          raw_file_list = sample_file_name,
                                          crop_num = None,
                                          crop_size = 1000)

In [109]:
rgb_sample = (rgb_sample + 1) * 200
rgb_sample = (np.minimum(rgb_sample, 2) * 127.5).astype(np.uint8)
rgb_sample_out_filename = training_dir + os.path.sep + 'rgb_sample.png'
if not os.path.exists(rgb_sample_out_filename):
    tl.vis.save_images(rgb_sample, [4, 4], training_dir + os.path.sep + 'rgb_sample.png')

In [99]:
rgb_sample.shape

(250, 250, 3)

In [84]:
rgb_sample[11].min()

-1.0

In [30]:
tl.vis.save_images?

In [16]:
train_data_list = tl.files.load_file_list(train_data_dir, regx = '.*.arw', printable = False)

In [17]:
for epoch in range(0, 1 + 1):
    epoch_time = time.time()
    total_mse_loss, n_iter = 0, 0

    for idx in range(0, len(train_data_list), batch_size):
        step_time = time.time()
        
        batch_file_name = train_data_list[idx: idx+batch_size]
        inputs_rgbs, label_raws = get_inputs_labels(train_data_dir, batch_file_name, crop_num)
        
        ## update G
        errM, _ = sess.run([mse_loss, g_optim_init], {t_image: inputs_rgbs, t_target_image: label_raws})
        print("Epoch [%2d/%2d] %4d time: %4.4fs, mse: %.8f " % (epoch, n_epoch_init, n_iter, time.time() - step_time, errM))
        total_mse_loss += errM
        n_iter += 1
    log = "[*] Epoch: [%2d/%2d] time: %4.4fs, mse: %.8f" % (epoch, n_epoch_init, time.time() - epoch_time, total_mse_loss / n_iter)
    print(log)

    ## quick evaluation on train set
#     if (epoch != 0) and (epoch % 10 == 0):
#         out = sess.run(net_g_test.outputs, {t_image: sample_imgs_96})  #; print('gen sub-image:', out.shape, out.min(), out.max())
#         print("[*] save images")
#         tl.vis.save_images(out, [ni, ni], save_dir_ginit + '/train_%d.png' % epoch)

    ## save model
    if (epoch != 0) and (epoch % 10 == 0):
        tl.files.save_npz(net_g.all_params, name=training_dir + '/g_srgan_init.npz', sess=sess)

Epoch [ 0/100]    0 time: 6.4022s, mse: 0.84584844 
Epoch [ 0/100]    1 time: 1.0244s, mse: 0.79371977 
Epoch [ 0/100]    2 time: 1.0310s, mse: 0.66120279 
Epoch [ 0/100]    3 time: 1.0831s, mse: 0.83286923 
[*] Epoch: [ 0/100] time: 9.5594s, mse: 0.78341006
Epoch [ 1/100]    0 time: 1.0732s, mse: 0.77754289 
Epoch [ 1/100]    1 time: 1.0275s, mse: 0.55351317 
Epoch [ 1/100]    2 time: 1.0779s, mse: 0.67140925 
Epoch [ 1/100]    3 time: 1.0318s, mse: 0.75701976 
[*] Epoch: [ 1/100] time: 4.2154s, mse: 0.68987127


In [16]:
for epoch in range(0, 2 + 1):
        ## update learning rate
        if epoch != 0 and (epoch % decay_every == 0):
            new_lr_decay = lr_decay**(epoch // decay_every)
            sess.run(tf.assign(lr_v, lr_init * new_lr_decay))
            log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
            print(log)
        elif epoch == 0:
            sess.run(tf.assign(lr_v, lr_init))
            log = " ** init lr: %f  decay_every_init: %d, lr_decay: %f (for GAN)" % (lr_init, decay_every, lr_decay)
            print(log)

        epoch_time = time.time()
        total_d_loss, total_g_loss, n_iter = 0, 0, 0

        for idx in range(0, len(train_data_list), batch_size):
            step_time = time.time()
            batch_file_name = train_data_list[idx: idx+batch_size]
            inputs_rgbs, label_raws = get_inputs_labels(train_data_dir, batch_file_name, 2)
            ## update D
            errD, _ = sess.run([d_loss, d_optim], {t_image: inputs_rgbs, t_target_image: label_raws})
            ## update G
            errG, errM, errA, _ = sess.run([g_loss, mse_loss, g_gan_loss, g_optim],
                                                 {t_image: inputs_rgbs, t_target_image: label_raws})
            print("Epoch [%2d/%2d] %4d time: %4.4fs, d_loss: %.8f g_loss: %.8f (mse: %.6f adv: %.6f)" %
                  (epoch, n_epoch, n_iter, time.time() - step_time, errD, errG, errM, errA))
            total_d_loss += errD
            total_g_loss += errG
            n_iter += 1

        log = "[*] Epoch: [%2d/%2d] time: %4.4fs, d_loss: %.8f g_loss: %.8f" % (epoch, n_epoch, time.time() - epoch_time, total_d_loss / n_iter,
                                                                                total_g_loss / n_iter)
        print(log)

        ## quick evaluation on train set
#         if (epoch != 0) and (epoch % 10 == 0):
#             out = sess.run(net_g_test.outputs, {t_image: sample_imgs_96})  #; print('gen sub-image:', out.shape, out.min(), out.max())
#             print("[*] save images")
#             tl.vis.save_images(out, [ni, ni], save_dir_gan + '/train_%d.png' % epoch)

        ## save model
        if (epoch != 0) and (epoch % 10 == 0):
            tl.files.save_npz(net_g.all_params, name=training_dir + '/g_srgan.npz'.format(tl.global_flag['mode']), sess=sess)
            tl.files.save_npz(net_d.all_params, name=training_dir + '/d_srgan.npz'.format(tl.global_flag['mode']), sess=sess)

 ** init lr: 0.000100  decay_every_init: 1000, lr_decay: 0.100000 (for GAN)
Epoch [ 0/2000]    0 time: 1.5955s, d_loss: 2.13979721 g_loss: 0.03410675 (mse: 0.000407 adv: 0.033700)
Epoch [ 0/2000]    1 time: 1.5941s, d_loss: 2.55675888 g_loss: 0.00868321 (mse: 0.000189 adv: 0.008494)
Epoch [ 0/2000]    2 time: 1.5318s, d_loss: 2.59092546 g_loss: 0.02898122 (mse: 0.006354 adv: 0.022627)
Epoch [ 0/2000]    3 time: 1.4577s, d_loss: 2.33331823 g_loss: 0.00952866 (mse: 0.000309 adv: 0.009220)
[*] Epoch: [ 0/2000] time: 6.1793s, d_loss: 2.40519994 g_loss: 0.02032496
Epoch [ 1/2000]    0 time: 1.5098s, d_loss: 1.96535134 g_loss: 0.01540101 (mse: 0.000368 adv: 0.015033)
Epoch [ 1/2000]    1 time: 1.5604s, d_loss: 0.69386184 g_loss: 0.01694207 (mse: 0.003474 adv: 0.013468)
Epoch [ 1/2000]    2 time: 1.5769s, d_loss: 1.97620749 g_loss: 0.03274266 (mse: 0.018687 adv: 0.014056)
Epoch [ 1/2000]    3 time: 1.4568s, d_loss: 7.11636019 g_loss: 0.00418333 (mse: 0.000659 adv: 0.003524)
[*] Epoch: [ 1/200